# Module 4: Types of Quantum Errors – Physics and Math Models

In the "Noise Awareness" component of the AQC challenge, you are encouraged to estimate a "latent noise descriptor" $z$. To do this, you must distinguish between different physical error channels.

## 4.1 The Physics of Failure

1.  **$T_1$ (Amplitude Damping):** The qubit "gets tired" and falls from $|1\rangle \to |0\rangle$ by releasing energy (photon) into the environment.
    * *Physics:* Coupling to high-frequency dielectric defects.
    * *Math:* Non-unital channel. Shrinks the Bloch sphere towards the North Pole ($|0\rangle$).

2.  **$T_2$ (Phase Damping):** The qubit "gets dizzy." It stays in the xy-plane but loses track of its rotation angle.
    * *Physics:* Low-frequency flux noise moving energy levels up and down.
    * *Math:* Shrinks the Bloch sphere towards the Z-axis.

3.  **Readout Error:** The camera is blurry. You measure a 0, but it was actually a 1.
    * *Math:* Modeled by a classical Confusion Matrix $\Lambda$.
    $$ \vec{P}_{observed} = \Lambda \vec{P}_{ideal} $$

## 4.2 Implementation: Building a Custom Noise Model

We will use `qiskit_aer` to create a realistic noise environment. This allows us to train our AI on "synthetic data" that looks exactly like the competition data.

In [7]:
from qiskit_aer.noise import NoiseModel, thermal_relaxation_error, ReadoutError
from qiskit_aer import AerSimulator
from qiskit import QuantumCircuit, transpile
import numpy as np

def build_noise_model(t1=50e-6, t2=70e-6, time_1q=50e-9, time_2q=400e-9, readout_err=0.05):
    """
    Constructs a noise model mimicking a real IBM device.

    Args:
        t1, t2: Relaxation and Dephasing times (seconds)
        time_1q: Duration of single-qubit gates (seconds) - typically fast (~50ns)
        time_2q: Duration of two-qubit gates (seconds) - typically slow (~400ns)
        readout_err: Probability of measurement flip
    """
    noise_model = NoiseModel()

    # --- 1. Thermal Relaxation Errors ---

    # A: Create the error for 1-qubit gates (fast)
    error_1q = thermal_relaxation_error(t1, t2, time_1q)
    noise_model.add_all_qubit_quantum_error(error_1q, ['sx', 'x', 'id', 'rz'])

    # B: Create the error for 2-qubit gates (slow)
    # Since 2-qubit gates involve 2 qubits, the error is the tensor product of two 1-qubit errors
    # Note: We use time_2q because the qubits sit idle/interacting for longer!
    error_2q_single = thermal_relaxation_error(t1, t2, time_2q)
    error_2q = error_2q_single.expand(error_2q_single) # Tensor product: error on q0 * error on q1

    noise_model.add_all_qubit_quantum_error(error_2q, ['ecr', 'cx'])

    # --- 2. Readout Error ---
    probabilities = [
        [1 - readout_err, readout_err],  # P(0|0), P(1|0)
        [readout_err, 1 - readout_err]   # P(0|1), P(1|1)
    ]
    readout_error = ReadoutError(probabilities)
    noise_model.add_all_qubit_readout_error(readout_error)

    return noise_model

# --- Test the Noise Model ---
# Create a circuit that should be |1>
qc = QuantumCircuit(1)
qc.x(0)
qc.measure_all()

# Simulate with Noise
noise_model = build_noise_model(readout_err=0.10) # 10% readout error
sim_noise = AerSimulator(noise_model=noise_model)

# We must transpile the circuit to the basis gates supported by the noise model
qc_noisy = transpile(qc, sim_noise)
result = sim_noise.run(qc_noisy, shots=1000).result()
counts = result.get_counts()

print(f"Noisy Counts (Expect mostly '1'): {counts}")

Noisy Counts (Expect mostly '1'): {'1': 892, '0': 108}
